<a href="https://colab.research.google.com/github/BxMild/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

#

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_house = pd.read_csv('/content/drive/MyDrive/Data_BSC_DPDM24/Data House.csv')
data_house.head

<bound method NDFrame.head of           id                                        name_th  \
0          1                        บ้านเดี่ยว ชนบท ขอนแก่น   
1          4                      บ้านเดี่ยว น้ำพอง ขอนแก่น   
2         16                   บ้านเดี่ยว มัญจาคีรี ขอนแก่น   
3         18          บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4         19       บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   
...      ...                                            ...   
1651  20460P  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026   
1652  20462P  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033   
1653  20463P  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581   
1654  20466P  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402   
1655  30358P  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-08537   

                                            name_en    project_name  \
0                           บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                         บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
2                      บ้านเดี่ยว มัญจาคีรี ขอนแก่น             NaN   
3             บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4          บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   
...                                             ...             ...   
1651  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027             NaN   
1652  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034             NaN   
1653  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582             NaN   
1654  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403             NaN   
1655  ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-08538             NaN   

                                    company province      district  \
0                                       NaN  ขอนแก่น          ชนบท   
1                                       NaN  ขอนแก่น        น้ำพอง   
2                                       NaN  ขอนแก่น     มัญจาคีรี   
3     บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน)  ขอนแก่น  เมืองขอนแก่น   
4               เบสท์โฮม เลี่ยงเมืองขอนแก่น  ขอนแก่น  เมืองขอนแก่น   
...                                     ...      ...           ...   
1651                                    NaN  ขอนแก่น       บ้านฝาง   
1652                                    NaN  ขอนแก่น      หนองเรือ   
1653                                    NaN  ขอนแก่น       แวงน้อย   
1654                                    NaN  ขอนแก่น         ชุมแพ   
1655                                    NaN  ขอนแก่น       บ้านไผ่   

     sub_district property_type home improvement  ... furniture  \
0     ศรีบุญเรือง    บ้านเดี่ยว               no  ...     ไม่มี   
1         ทรายมูล    บ้านเดี่ยว               no  ...     ไม่มี   
2         กุดเค้า    บ้านเดี่ยว               no  ...   ไม่ระบุ   
3        บ้านเป็ด    บ้านเดี่ยว               no  ...   ไม่ระบุ   
4        บ้านเป็ด    บ้านเดี่ยว               no  ...   ไม่ระบุ   
...           ...           ...              ...  ...       ...   
1651      หนองบัว    บ้านเดี่ยว               no  ...   ไม่ระบุ   
1652       โนนทอง    บ้านเดี่ยว               no  ...   ไม่ระบุ   
1653   ก้านเหลือง    บ้านเดี่ยว               no  ...   ไม่ระบุ   
1654    วังหินลาด    บ้านเดี่ยว               no  ...   ไม่ระบุ   
1655      หินตั้ง    บ้านเดี่ยว               no  ...   ไม่ระบุ   

     building_number number_of_building  floor_number  parking_percentage  \
0                NaN                NaN           NaN                 NaN   
1                NaN                NaN           NaN                 NaN   
2                NaN                NaN           NaN                 NaN   
3                NaN                NaN           NaN                 NaN   
4                NaN                NaN           NaN                 NaN   
...              ...                ...           ...                 ...   
1651             NaN                NaN           NaN                 NaN   
1652             NaN                NaN           NaN                 NaN   
1653             NaN 

In [4]:
appraisal = pd.read_csv('/content/drive/MyDrive/Data_BSC_DPDM24/คำตอบนักประเมิน(ID).csv')
appraisal.head

<bound method NDFrame.head of       ID ผู้ประเมิน                                post id     ID  \
0                 1     https://nayoo.co/khonkaen/posts/25     25   
1                 2     https://nayoo.co/khonkaen/posts/25     25   
2                 1     https://nayoo.co/khonkaen/posts/33     33   
3                 2     https://nayoo.co/khonkaen/posts/33     33   
4                 1    https://nayoo.co/khonkaen/posts/495    495   
...             ...                                    ...    ...   
1912              2  https://nayoo.co/khonkaen/posts/17867  17867   
1913              2  https://nayoo.co/khonkaen/posts/17873  17873   
1914              2  https://nayoo.co/khonkaen/posts/17874  17874   
1915              2  https://nayoo.co/khonkaen/posts/17876  17876   
1916              2  https://nayoo.co/khonkaen/posts/17885  17885   

     property_type ความมั่นใจในการเมินราคาทรัพย์  ราคาทรัพย์  \
0       บ้านเดี่ยว                        มั่นใจ   2,890,000   
1       บ้านเดี่ยว                        มั่นใจ   2,890,000   
2       บ้านเดี่ยว                        มั่นใจ   2,600,000   
3       บ้านเดี่ยว                        มั่นใจ   2,600,000   
4       บ้านเดี่ยว                        มั่นใจ   6,050,000   
...            ...                           ...         ...   
1912    บ้านเดี่ยว                        มั่นใจ     965,000   
1913    บ้านเดี่ยว                        มั่นใจ   1,000,000   
1914    บ้านเดี่ยว                        มั่นใจ  10,873,000   
1915    บ้านเดี่ยว                        มั่นใจ   1,000,000   
1916    บ้านเดี่ยว                        มั่นใจ   3,200,000   

     ประเมินราคาซื้อ-ขาย                     ปัจจัยในการประเมินราคาซื้อ-ขาย  \
0              2,500,000                                       5,000-15,000   
1              2,700,000  โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...   
2              2,200,000                                       5,000-15,000   
3              2,400,000   ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง   
4              5,000,000                                       5,000-15,000   
...                  ...                                                ...   
1912             550,000              ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา   
1913           1,000,000              ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา   
1914          10,000,000            ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา   
1915           1,200,000                   ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง   
1916           3,500,000                   ราคาเสนอขาย 3.5-5.0 ล้านบาท/หลัง   

     เหตุผลไม่มั่นใจราคาประเมิน                      ให้คะแนนทรัพย์  
0                           NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
1                           NaN    4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน  
2                           NaN  2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี  
3                           NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
4                           NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
...                         ...                                 ...  
1912                        NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
1913                        NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
1914                        NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
1915                        NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
1916                        NaN   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  

[1917 rows x 10 columns]>

In [5]:
# prompt: read json file in drive

import json

# Load the JSON data from the file in Google Drive
with open('/content/drive/MyDrive/Data_BSC_DPDM24/7-eleven.json', 'r') as f:
    seven_eleven_data = json.load(f)

with open('/content/drive/MyDrive/Data_BSC_DPDM24/school.json', 'r') as f:
    school_data = json.load(f)

# Now you can work with seven_eleven_data and school_data
# Example: Print the first few entries
print(list(seven_eleven_data.keys())[:3])
print(list(school_data.keys())[:3])

['1', '4', '6']
['1', '4', '6']


## จัดการ data_house

### 1. ตรวจสอบและจัดการข้อมูล missing

In [6]:
# ใช้ fillna โดยไม่ใช้ inplace=True
data_house['number_of_bedroom'] = data_house['number_of_bedroom'].fillna(data_house['number_of_bedroom'].mean())
data_house['usable_area'] = data_house['usable_area'].fillna(data_house['usable_area'].mean())
data_house['sell_price'] = data_house['sell_price'].fillna(data_house['sell_price'].mean())
data_house['number_of_parking'] = data_house['number_of_parking'].fillna(data_house['number_of_parking'].mean())
data_house['renovate'] = data_house['renovate'].fillna(data_house['renovate'].mode()[0])
data_house['number_of_story'] = data_house['number_of_story'].fillna(data_house['number_of_story'].mode()[0])

### 2. การกำจัดค่า outliers ด้วย IQR (Interquartile Range)

In [7]:
# คำนวณ IQR สำหรับคอลัมน์ที่เกี่ยวข้อง
Q1 = data_house[['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']].quantile(0.25)
Q3 = data_house[['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']].quantile(0.75)
IQR = Q3 - Q1

# กำจัด outliers
data_house_cleaned = data_house[~((data_house[['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']] < (Q1 - 1.5 * IQR)) | (data_house[['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']] > (Q3 + 1.5 * IQR))).any(axis=1)]


### 3. การทำ Normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_house_cleaned.loc[:, ['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']] = scaler.fit_transform(data_house_cleaned[['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']])

<ipython-input-8-9c8b67e1260b>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5 0.  1.  ... 0.  0.  0. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_house_cleaned.loc[:, ['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']] = scaler.fit_transform(data_house_cleaned[['number_of_bedroom', 'number_of_bathroom', 'usable_area', 'sell_price']])


### 4. การทำ One-Hot Encoding สำหรับคอลัมน์ที่เป็น Categorical

In [9]:
data_house_encoded = pd.get_dummies(data_house_cleaned, columns=['direction', 'furniture', 'zone'], drop_first=True)

In [10]:
data_house = data_house_encoded

## คำนวณจำนวน 7-11 และโรงเรียนในรัศมี 1 กิโลเมตร

In [37]:
!pip install haversine
import haversine

In [38]:
def count_nearby(lat, lon, places_data):
    count = 0
    for place_id, places in places_data.items():
        for place in places:
            place_lat = place['geometry']['location']['lat']  # ใช้ 'geometry' -> 'location' -> 'lat'
            place_lon = place['geometry']['location']['lng']  # ใช้ 'geometry' -> 'location' -> 'lng'
            distance = haversine(lat, lon, place_lat, place_lon)
            if distance <= 1:  # ถ้าอยู่ในรัศมี 1 กม.
                count += 1
    return count

# คำนวณจำนวน 7-11 และโรงเรียนในรัศมี 1 กิโลเมตร
data_house['d1000-7/11'] = data_house.apply(lambda row: count_nearby(row['latitude'], row['longitude'], seven_eleven_data), axis=1)
data_house['d1000-school'] = data_house.apply(lambda row: count_nearby(row['latitude'], row['longitude'], school_data), axis=1)


TypeError: 'module' object is not callable

## ประเมินการซื้อขาย

In [18]:
# ฟังก์ชั่นในการตรวจสอบว่า ID ของบ้านอยู่ในรายการของ 7-11 หรือ โรงเรียน
def check_nearby(row_id, data_list):
    return 1 if row_id in data_list else 0

In [19]:
data_house = data_house.merge(
    appraisal[['ID', 'ประเมินราคาซื้อ-ขาย', 'ให้คะแนนทรัพย์']],
    left_on='id', right_on='ID', how='left'
)

# เปลี่ยนชื่อคอลัมน์ให้ตรงกับความต้องการ
data_house.rename(columns={
    'ประเมินราคาซื้อ-ขาย': 'estimator',
    'ให้คะแนนทรัพย์': 'score'
}, inplace=True)


In [20]:
# แปลงคอลัมน์ 'estimator' และ 'score' เป็นตัวเลข (float) ก่อนการคำนวณ
data_house['estimator'] = pd.to_numeric(data_house['estimator'], errors='coerce')
data_house['score'] = pd.to_numeric(data_house['score'], errors='coerce')

# คำนวณราคาโดยเฉลี่ยจากราคาประเมิน
data_house['predicted_price'] = data_house['estimator'].mean()

# คำนวณคะแนนทรัพย์
data_house['property_score'] = data_house['score'].mean()

# หากต้องการดูผลลัพธ์ที่คำนวณแล้ว
print(data_house[['id', 'predicted_price', 'property_score']].head())


   id  predicted_price  property_score
0   1        2898000.0             NaN
1   1        2898000.0             NaN
2   4        2898000.0             NaN
3   4        2898000.0             NaN
4  18        2898000.0             NaN


In [21]:
# คำนวณราคาโดยเฉลี่ยจากราคาประเมิน
data_house['predicted_price'] = data_house['estimator'].mean()

# คำนวณคะแนนทรัพย์
data_house['property_score'] = data_house['score'].mean()

# หากต้องการดูผลลัพธ์ที่คำนวณแล้ว
print(data_house[['id', 'predicted_price', 'property_score']].head())


   id  predicted_price  property_score
0   1        2898000.0             NaN
1   1        2898000.0             NaN
2   4        2898000.0             NaN
3   4        2898000.0             NaN
4  18        2898000.0             NaN


# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [22]:
print(data_house.columns)


Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'alley', 'street',
       'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'building_number', 'number_of_building', 'floor_number',
       'parking_percentage', 'has_pool', 'has_keycard', 'has_security',
       'has_fitness', 'has_garden', 'direction_ตะวันตกเฉียงเหนือ',
       'direction_ตะวันตกเฉียงใต้', 'direction_ตะวันออก',
       'direction_ตะวันออกเฉียงเหนือ', 'direction_ตะวันออกเฉียงใต้',
       'direction_ตะวันออกเฉียงใต้ของเมือง', 'direction_ทางทิศตะวันออก',
       'direction_ทิศตะวันออก', '

In [30]:
# คำนวณราคาเฉลี่ยของ 'sell_price' ตาม zone ที่มีคอลัมน์ที่เริ่มต้นด้วย 'zone_'
zone_columns = [col for col in data_house.columns if col.startswith('zone_')]
zone_data = data_house[zone_columns + ['sell_price']]

# คำนวณค่าเฉลี่ยของ 'sell_price' สำหรับแต่ละโซน
zone_avg_price = zone_data.mean()

# หาค่า zone ที่มีราคาเฉลี่ยสูงสุด
highest_zone = zone_avg_price.idxmax()
highest_price = zone_avg_price.max()

# แสดงชื่อโซนที่มีราคาเฉลี่ยสูงสุด
zone_name = highest_zone.split('_')[-1]
highest_zone, highest_price, zone_name

('sell_price', 0.33749133155785255, 'price')

In [27]:
# หาบ้านที่มีจำนวน 7-11 ในรัศมี 1 กม. มากที่สุด
max_seven_eleven_count = data_house['d1000-7/11'].max()
house_with_max_seven_eleven = data_house[data_house['d1000-7/11'] == max_seven_eleven_count]

# ดูราคาของบ้านที่มีจำนวน 7-11 มากที่สุด
house_with_max_seven_eleven_price = house_with_max_seven_eleven['sell_price'].values[0]

max_seven_eleven_count, house_with_max_seven_eleven_price


(3209, 0.30522341095028316)

In [39]:
# prompt: ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด

# Group data by sub_district and calculate the mean score for each sub_district
sub_district_avg_score = data_house.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest average score
sub_district_with_highest_score = sub_district_avg_score.idxmax()
highest_avg_score = sub_district_avg_score.max()

print(f"ตำบลที่มีคะแนนทรัพย์เฉลี่ยสูงสุดคือ: {sub_district_with_highest_score} (คะแนนเฉลี่ย: {highest_avg_score})")

ตำบลที่มีคะแนนทรัพย์เฉลี่ยสูงสุดคือ: nan (คะแนนเฉลี่ย: nan)


<ipython-input-39-c60540908f03>:7: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  sub_district_with_highest_score = sub_district_avg_score.idxmax()
